# Introduction

Dans cette introduction nous allons donner un très court aperçu des possibilités d'utilisation du langage Python pour l'économiste. Pour cela:
- nous allons présenter des exemples de simulation de modèles simples de l'économie,
- des exemples du travail empirique pouvant être fait.

**Résumé:** Un notebook Jupyter est un document composé de **cellules** qui contiennent:
- Le code de programmation qui dans notre cas sera du code Python.
- Du texte dont du texte mathématiques écrit dans le "style" de LaTex. Ces cellules sont des cellules *markdown*.
- Des résultats issus de l'execution du code programmé.

Ainsi, ceci est une cellule avec du texte et plus précisément cellule *markdown*.

* Appuyer sur <kbd>Enter</kbd> pour *éditer* la cellule.
* Appuyer sur <kbd>Ctrl + Enter</kbd> to *executer* la cellule.
* Appyer sur <kbd>Shift + Enter</kbd> to *executer* la cellule et avancer.

On peut y écrire sous forme de listes:

1. **Première** entrée.
2. *Deuxième* entrée.
3. ~~Troisième~~ item entrée barrée.

On peut écrire de maths, e.g. $\alpha^2$ ou

$$
X = \int_0^{\infty} \frac{x}{x+1} \mathrm{d}x
$$

In [ ]:
# Ceci est une cellule de code
x = 10   # Ce qui est écrit après un "#" est du "commentaire" du code, il n'est donc pas exécuté

# Procédons à des calculs
a = 20
b = 30
c = a + b

# Affichons les résultats (au-dessous de cette cellule)
print(c)

On peut comme ici écrire un peu plus de texte(cellule *markdown*) et poursuivre les calculs(cellules de code).

In [ ]:
d = c * 2
print(d)
print(c)

**Remarque:** 

Bien que JupyterLab/Jupyter s'execute avec un serveur, cela se fait sans besoin de connexion web (le chemin étant quelque chose comme *localhost:8888/lab*).<br>

**Binder:** Une exception est d'utiliser [binder](https://mybinder.org), dans ce cas JupyterLab s'executera sur un cloud.

# Résoudre le problème du consommateur.

On considère ici l'exemple classique des manuels de cours de Microéconomie que vous avez vu dans vos études.

Le problème est le suivant:

$$
\begin{align*}
V(p_{1},p_{2},I) & = \max_{q_{1},q_{2}} q_{1}^{\alpha}q_{2}^{1-\alpha}\\
 & \text{s.c.}\\
p_{1}q_{1}+p_{2}q_{2} & \leq R, \quad p_{1}, p_{2}, R > 0, \quad q_{1},q_{2}  \geq 0
\end{align*}
$$

On peut résoudre ce problème **numériquement**.

1. Choix de **paramètres**:

In [ ]:
alpha = 0.5
R = 10
p1 = 1
p2 = 2

2. La **fonction objectif** du consommateur est:

In [ ]:
# On définit la fonction objectif.
def val_cons(q1, alpha, R, p1, p2): 
    
    # a. Tout le revenu n'est pas dépensé sur le premier bien
    #    Et ce qui est dépensé dans le deuxième est:
    q2 = (R - p1*q1) / p2 
    
    # b. L'utilité obtenue est:
    util = q1**alpha*q2**(1-alpha)
    
    return util # résultat produit par la fonction

3. On utilise une fonction du module *scipy* pour **résoudre le problème**.

In [ ]:
# a. On importe à partir du module scipy la fonction optimize
from scipy import optimize

# b. la valeur du choix comme seulement fonction de q1
obj = lambda q1: -val_cons(q1, alpha, R, p1, p2)

# c. on appelle la fonction minimisatrice
solution = optimize.minimize_scalar(obj, bounds=(0, R/p1))

# d. affichage des résultats
q1 = solution.x
q2 = (R - q1*p1) / p2
print(q1, q2)

**Tâche**: résoudre le même problème avec la fonction d'utilité CES.

$$
u(q_1,q_2) = (\alpha q_1^{-\beta} + (1-\alpha) x_2^{-\beta})^{-1/\beta}
$$

In [ ]:
# a. choix des paramètres
alpha = 0.5
beta = 0.000001
R = 10
p1 = 1
p2 = 2

# b. valeur du choix
def val_cons_ces(q1, alpha, beta, R, p1, p2):
    q2 = (R - p1*q1)/p2
    if q1 > 0 and q2 > 0:
        util = (alpha*q1**(-beta) + (1 - alpha)*q2**(-beta))**(-1 / beta) 
    else:
        util = 0
    return util

# c. objectif
obj = lambda q1: -val_cons_ces(q1, alpha, beta, R, p1, p2)

# d. solution
solution = optimize.minimize_scalar(obj, bounds=(0, R/p1))

# e. résultat
q1 = solution.x
q2 = (R - q1 * p1)/p2
print(q1, q2)

# Simulation du modèle AS-AD

**Remarque**: c'est un modèle macroéconomique pour [l'offre et la demande globales dans une économie](https://fr.wikipedia.org/wiki/Mod%C3%A8le_OG-DG).

Soit le modèle défini par:

$$
\begin{align*}
\hat{y}_{t} &= b\hat{y}_{t - 1}+\beta(z_{t} - z_{t-1}) -a\beta s_{t} + a\beta\phi s_{t-1} \\
\hat{\pi}_{t} &= b\hat{\pi}_{t - 1}+\beta\gamma z_{t} - \beta\phi\gamma z_{t}+\beta s_{t} - \beta\phi s_{t-1} \\
z_{t} &= \delta z_{t - 1}+x_{t}, x_{t} \sim N(0, \sigma_x^2) \\
s_{t} &= \omega s_{t - 1}+c_{t}, c_{t} \sim N(0, \sigma_c^2) \\
b &= \frac{1 + a\phi\gamma}{1 + a\gamma} \\
\beta &= \frac{1}{1 + a\gamma}
\end{align*}
$$

où $\hat{y}_{t}$ est l'écart de production, $\hat{\pi}_{t}$ est l'écart d'inflation, $z_{t}$ est un choc de demande AR(1), et $\hat{s}_{t}$ est un chox d'offre AR(1).

1. Choix de **paramètres**:

In [ ]:
a = 0.4
gamma = 0.1
phi = 0.9
delta = 0.8
omega = 0.15
sigma_x = 1
sigma_c = 0.4
T = 100

2. Calcul des **paramètres composites**:

In [ ]:
b = (1 + a*phi*gamma)/(1 + a*gamma)
beta = 1/(1 + a*gamma) 

3. Définition des **fonctions du modèle**:

In [ ]:
y_hat_func = lambda y_hat_lag, z,z_lag, s, s_lag: b*y_hat_lag + beta*(z-z_lag) - a*beta*s + a*beta*phi*s_lag
pi_hat_func = lambda pi_lag,z,z_lag,s,s_lag: b*pi_lag + beta*gamma*z - beta*phi*gamma*z_lag + beta*s - beta*phi*s_lag
z_func = lambda z_lag,x: delta*z_lag + x
s_func = lambda s_lag,c: omega*s_lag + c

4. On execute la **simulation**:

In [ ]:
import numpy as np # importation du module numpy

# a. mise en place du générateur de nombre aléatoires
np.random.seed(2015)   

# b. affectation des données simulées
x = np.random.normal(loc=0, scale=sigma_x, size=T)
c = np.random.normal(loc=0, scale=sigma_c, size=T)
z = np.zeros(T)
s = np.zeros(T)
y_hat = np.zeros(T)
pi_hat = np.zeros(T)

# c. execution de la simulation
for t in range(1,T):

    # i. actualisation de z et s
    z[t] = z_func(z[t-1], x[t])
    s[t] = s_func(s[t-1], c[t])

    # ii. calcul de y og pi 
    y_hat[t] = y_hat_func(y_hat[t-1], z[t], z[t-1], s[t], s[t-1])
    pi_hat[t] = pi_hat_func(pi_hat[t-1], z[t], z[t-1], s[t], s[t-1])

5. **Visualisation** de la simulation:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.plot(y_hat, label='$\\hat{y}$')
ax.plot(pi_hat, label='$\\hat{\pi}$')

ax.set_xlabel('temps')

ax.set_ylabel('pourcentage')
ax.set_ylim([-8,8])

ax.legend(loc='upper left');

On peut aussi utiliser **le style seaborn**:

In [ ]:
plt.style.use('seaborn-whitegrid')

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.plot(y_hat,label='$\\hat{y}$')
ax.plot(pi_hat,label='$\\hat{pi}$')

ax.set_xlabel('temps')

ax.set_ylabel('poucentage')
ax.set_ylim([-8,8])

ax.legend(loc='upper left',facecolor='white',frameon='True');

# Modules

Un **module** est un fichier **.py**-file avec des fonctions qu'on importe et auxquelle on peut alors faire appel dans le noebook.

Try to open **mymodule.py** and have a look.

In [20]:
import monmodule

In [21]:
x = 5
y = monmodule.mafonction(x)
print(y)

25


# Analyse de données

## Données sur COVID 19 au Pérou

Les données proviennent de la source la [Plateforme National de Données Ouvertes](https://www.datosabiertos.gob.pe/) du Pérou. Plus de détail peuvent être obtenus [ici](https://www.datosabiertos.gob.pe/dataset/hospitalizados-vacunados-y-fallecidos-por-covid-19).

In [27]:
import pandas as pd # On importe la bibliothèque pandas
df = pd.read_csv('/Users/michalurdanivia/Documents/Temp_Paris_11072021_etc/TB_HOSP_VAC_FALLECIDOS.csv')
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117126 entries, 0 to 117125
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   eess_renaes                   117126 non-null  int64  
 1   eess_diresa                   117126 non-null  object 
 2   eess_red                      117126 non-null  object 
 3   eess_nombre                   117126 non-null  object 
 4   id_eess                       117126 non-null  int64  
 5   id_persona                    117126 non-null  int64  
 6   edad                          111482 non-null  float64
 7   sexo                          112541 non-null  object 
 8   fecha_ingreso_hosp            117126 non-null  object 
 9   flag_uci                      117126 non-null  int64  
 10  fecha_ingreso_uci             10585 non-null   object 
 11  fecha_ingreso_ucin            5659 non-null    object 
 12  con_oxigeno                   117121 non-nul

In [28]:
# Description basique des données(seulement des moyennes, écart-types, et quantiles)
df.describe()

,eess_renaes,id_eess,id_persona,edad,flag_uci,con_oxigeno,con_ventilacion,flag_vacuna,cdc_positividad,cdc_fallecido_covid,ubigeo_inei_domicilio
count,117126.000000,117126.000000,1.171260e+05,111482.000000,117126.000000,117121.000000,117112.000000,117126.000000,117126.000000,117126.000000,89031.000000
mean,5906.226628,5767.677151,1.948864e+07,51.941614,0.125677,0.728674,0.092561,0.762060,0.557801,0.297782,123805.079377
std,5412.769415,5055.547865,1.280993e+07,19.755807,0.331486,0.444646,0.289817,0.904658,0.496650,0.457285,61426.453840
min,3.000000,3.000000,1.100000e+01,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10101.000000
25%,2289.000000,2288.000000,7.526137e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80104.000000
50%,5577.000000,5574.000000,1.803483e+07,53.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,150102.000000
75%,6218.000000,6215.000000,3.531059e+07,67.000000,0.000000,1.000000,0.000000,2.000000,1.000000,1.000000,150142.000000
max,28025.000000,26260.000000,3.784083e+07,110.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,250401.000000
